# Construction of the pYPKa vector
The pYPKa is a version of the pCAPs vector where a unique [AjiI](http://rebase.neb.com/rebase/enz/AjiI.html) site has been created by site directed mutagenesis and a [AcsI](http://rebase.neb.com/rebase/enz/AcsI.html) site removed. The pCAPs plasmid was described by [Schlieper et al. 1998](http://www.ncbi.nlm.nih.gov/pubmed/9514792):

In [1]:
# NBVAL_IGNORE_OUTPUT
from IPython.display import IFrame
IFrame('https://www.ncbi.nlm.nih.gov/pubmed/9514792', width='80%', height=250)

Importing the [pydna](https://pypi.python.org/pypi/pydna/) package.
Pydna is [open source](https://github.com/BjornFJohansson/pydna), documentated [here](http://pydna.readthedocs.org/en/latest/) and has a support [forum](https://groups.google.com/forum/#!forum/pydna) as well as a publication:

[Pereira F, Azevedo F, Carvalho Â, Ribeiro GF, Budde MW, Johansson B: Pydna: a simulation and documentation tool for DNA assembly strategies using python. BMC Bioinformatics 2015, 16:142.](http://www.biomedcentral.com/1471-2105/16/142/abstract)

In [2]:
from pydna.all import *

The plasmid backbone sequence comes from a Genbank [record](http://www.ncbi.nlm.nih.gov/nuccore/AJ001614).
Access to Genbank is needed in order to download the template.
If you execute this script, change the email address below to your own.
Always tell Genbank who you are, when using the service.

In [3]:
gb = Genbank("bjornjobbb@gmail.com")

In [4]:
pCAPs = gb.nucleotide("AJ001614.1")

In [5]:
assert pCAPs.cseguid() == "S6GoB_Z13VCfGDTEqNclu4It614"

We read the file from a local copy with more features defined.

In [6]:
pCAPs = read("pCAPs.gb")

In [7]:
assert pCAPs.cseguid() == "S6GoB_Z13VCfGDTEqNclu4It614"

The sequence to be mutated is contained withing the subsequence below:

In [8]:
print(str(pCAPs[555:604].seq))

TTCTCACTAGTGACCTGCAGCCGGCGCGCCATCTGTGCAGACAAACGCA


In [9]:
from Bio.Restriction import AjiI
print(AjiI.site)
print(AjiI.is_palindromic())

CACGTC
False


## Outline of the mutation strategy

pCAPs sequence

    TTCTCACTAGTGACCTGCAGCCGGCGCGCCATCTGTGCAGACAAACGCA   watson  
    AAGAGTGATCACTGGACGTCGGCCGCGCGGTAGACACGTCTGTTTGCGT   crick
                       <---ArgAla---

Two aminoacids in the CRP coding sequence are indicated in the crick strand of pCAPs.


Primers annealing to template
                            
    TTCTCACTAGTGACCTGCAGCCGGCGCGCCATCTGTGCAGACAAACGCA
      |||||||||||||||||||||
      gagtgatcactggacgtcggcTG
                             GTGCcatctgtgcagacaaacg
                                |||||||||||||||||||
    AAGAGTGATCACTGGACGTCGGCCGCGCGGTAGACACGTCTGTTTGCGT
    
Resulting mutated seqence in pYPKa:    
                                
    ttctcactagtgacctgcagccgACGTGccatctgtgcagacaaacgca   watson
    aagagtgatcactggacgtcggcTGCACggtagacacgtctgtttgcgt   crick
                          ------                        AjiI site
    
                       <---ArgAla---
    
    
The mutation is silent, the codons formed in the CRP orf on the crick strand encode the same aminoacids
    
          Ala Arg
    
    pCAPS gcg cgg
    
    pYPKa gca cgt
   

In [10]:
primers =parse('''
>568_pCAPsAjiIR (22-mer)
GTGCcatctgtgcagacaaacg
>567_pCAPsAjiIF (23-mer)
GTcggctgcaggtcactagtgag''', ds=False)

In [11]:
pYPKa =pcr(primers, pCAPs).looped().synced(pCAPs)

In [12]:
assert pYPKa.cseguid() == "aV1eIrzOiCjvw01yvKkxDXHKLMk"

In [13]:
pYPKa.stamp()

cSEGUID_aV1eIrzOiCjvw01yvKkxDXHKLMk

The first of the pCAPs features is the crp gene. This is left out in the pYPKa due to the PCR process. 
It is added back to the pYPKa features below.

In [14]:
crp_feature = pCAPs.features[0]

In [15]:
crp_feature.qualifiers["note"]

['artificially mutated crp; cyclic AMP receptor protein']

In [16]:
pYPKa.features.append(crp_feature)

In [17]:
from Bio.Restriction import EcoRV, ZraI, AjiI

In [18]:
pYPKa.name = "pYPKa"

In [19]:
pYPKa.write("pYPKa.gb")

pYPKa.gb